## Solution of the mixed integer linear pump scheduling formulation using IBM's CPLEX solver (commercial) and COINOR's CBC solver (free open source)

### T.Janus
### 21/05/2023

### README

The linear programme represented in a `.mps` format is read and solved with two solvers - a commercial **IBM ILOG CPLEX** solver requiring an academic/business license and an open-source free **COINOR CBC** solver. **CPLEX** solver is executd using the IBM's CPLEX Python binding. **CBC** solver is executed with COINOR's [PYTHON-MIP](https://www.python-mip.com/) library for the modeling and solution of Mixed-Integer Linear programs (MIPs) that supports creating large scale linear programmes and comes packaged with CBC binaries.

#### TODO:
1. Install and test solving the problem with Gurobi using PYTHON-MIP
2. Introduction os special ordered sets (SOS) https://docs.python-mip.com/en/latest/sos.html

### 1. OPTIMIZE WITH CPLEX

In [ ]:
import pathlib
import cplex
import mip
# import docplex.mp # Library for creating optimization models in Python
import scipy.io

In [ ]:
MODEL_MPS_FILE = pathlib.Path("../data/2p1t/2p1t.mps")
MODEL_MATRICES_FILE = pathlib.Path("../data/2p1t/2p1t_model.mat")
MODEL_LP_FILE = pathlib.Path("../data/2p1t/2p1t.lp")
CPLEX_RESULTS_FILE = pathlib.Path("../outputs/2p1t/x_optim_cplex.mat")
CBC_RESULTS_FILE = pathlib.Path("../outputs/2p1t/x_optim_cbc.mat")

In [ ]:
# Load matrices from Matlab (for testing purposes)
matlab_model = scipy.io.loadmat(MODEL_MATRICES_FILE)
# print(matlab_model['ub'])

In [ ]:
# Convert MPS file to LP file
instance = mip.Model()
instance.read(MODEL_MPS_FILE.as_posix())
instance.write(MODEL_LP_FILE.as_posix())

In [ ]:
# Create a CPLEX problem object
problem = cplex.Cplex()
# problem.parameters.lpmethod = 4
# Read the MPS file
problem.read(MODEL_MPS_FILE.as_posix())
# Set the problem type to MILP
problem.set_problem_type(cplex.Cplex.problem_type.MILP)
# Set the tolerance gap
problem.parameters.mip.tolerances.mipgap.set(0.001)
# Solve the problem
problem.solve()

In [ ]:
print(f"Solution status: {problem.solution.get_status_string()}")
print(f"Objective value: {problem.solution.get_objective_value()}")

In [ ]:
x_optim = problem.solution.get_values()
scipy.io.savemat(CPLEX_RESULTS_FILE, {"x": x_optim})

In [ ]:
print("Solution vector:")
for i, name in enumerate(problem.variables.get_names()):
    print(f"{name} = {problem.solution.get_values(i):.3f}")
    if i > 5:
        print("...")
        break

### 2. OPTIMIZE WITH CBC

In [ ]:
instance = mip.Model(solver_name=mip.CBC)
instance.read(MODEL_MPS_FILE.as_posix())
instance.write(MODEL_LP_FILE.as_posix())
# Set optimization parameters
instance.threads = -1
instance.max_gap = 0.15
#instance.cut_passes = 10
#instance.clique = -1  # -1 means automatic, 0 disables it, 1 enables it and 2 enables more aggressive clique generation
#instance.cuts = 2
#instance.emphasis = 2
#instance.infeas_tol = 1e-4
#instance.preprocess = 1
#instance.pump_passes = 10
instance.lp_method = 1
optim_status = instance.optimize(max_seconds=600)
if optim_status == mip.OptimizationStatus.OPTIMAL:
    print('\n Optimal solution cost {} found'.format(instance.objective_value))
elif optim_status == mip.OptimizationStatus.FEASIBLE:
    print('\n Feasible solution cost {} found, best possible: {}'.format(instance.objective_value, instance.objective_bound))
elif optim_status == mip.OptimizationStatus.NO_SOLUTION_FOUND:
    print('\n No feasible solution found, lower bound is: {}'.format(instance.objective_bound))

In [ ]:
instance.objective.x

In [ ]:
instance.objective_value

In [ ]:
x_optim_cbc = [var.x for var in instance.vars]
scipy.io.savemat(CBC_RESULTS_FILE, {"x": x_optim_cbc})

In [ ]:
#diff = [x1 - x2 for x1, x2 in zip(x_optim, x_optim_cbc)]
#diff